# Human-in-the-loop

When building workflows or agents with Bridgic, developers can seamlessly integrate human-in-the-loop interactions into the execution flow. At any point, the system can pause its automated process to request human input — such as approval, verification, or additional instructions — and wait for a response. Once the human feedback is provided, the workflow or agent resumes execution from the point of interruption, adapting its behavior based on the new input. Bridgic ensures that the entire process, including pause and resume states, can be reliably serialized and deserialized for persistence and recovery.

## Interaction Scenarios

Let's go through a few simple examples to understand this process. Before that, let's set up the running environment.

In [ ]:
import os

# Set the API base and key.
_api_key = os.environ.get("OPENAI_API_KEY")
_api_base = os.environ.get("OPENAI_API_BASE")
_model_name = os.environ.get("OPENAI_MODEL_NAME")

from pydantic import BaseModel, Field
from bridgic.core.automa import GraphAutoma, worker, Snapshot
from bridgic.core.automa.args import From
from bridgic.core.automa.interaction import Event, Feedback, FeedbackSender, InteractionFeedback, InteractionException
from bridgic.core.model.types import Message, Role
from bridgic.core.model.protocols import PydanticModel
from bridgic.llms.openai import OpenAILlm

### Programming assistant

During the development of a programming assistant, it can be designed to automatically execute and verify the code it generates. However, since program execution consumes system resources, the user must decide whether to grant permission for the assistant to run the code.

Let's achieve it with Bridgic. The steps are as follows:

1. Generate code based on user requirements.
2. Ask the user if it is allowed to execute the generated code.
3. Output result.

In [ ]:
# Set the LLM
llm = OpenAILlm(api_base=_api_base, api_key=_api_key, timeout=10)

class CodeBlock(BaseModel):
    code: str = Field(description="The code to be executed.")

class CodeAssistant(GraphAutoma):
    @worker(is_start=True)
    async def generate_code(self, user_requirement: str):
        response = await llm.astructured_output(
            model=_model_name,
            messages=[
                Message.from_text(text=f"You are a programming assistant. Please generate code according to the user's requirements.", role=Role.SYSTEM),
                Message.from_text(text=user_requirement, role=Role.USER),
            ],
            constraint=PydanticModel(model=CodeBlock)
        )
        return response.code

    @worker(dependencies=["generate_code"])
    async def ask_to_run_code(self, code: str):
        event = Event(event_type="can_run_code", data=code)
        feedback = await self.request_feedback_async(event)
        return feedback.data
        
    @worker(dependencies=["ask_to_run_code"])
    async def output_result(self, feedback: str, code: str = From("generate_code")):
        code = code.strip("```python").strip("```")
        if feedback == "yes":
            print(f"- - - - - - Result - - - - - -")
            exec(code)
            print(f"- - - - - - End - - - - - -")
        else:
            print(f"This code was rejected for execution. In response to the requirements, I have generated the following code:\n {code}")

In the `ask_to_run_code()` method of `CodeAssistant`, we use [`request_feedback_async()`](../../../../reference/bridgic-core/bridgic/core/automa/#bridgic.core.automa.Automa.request_feedback_async) to send an Event to the human user and expect to receive a feedback. To handle this Event, the corresponding logic needs to be registered with the automa, like this:

In [31]:
import getpass

# Handle can_run_code event
def can_run_code_handler(event: Event, feedback_sender: FeedbackSender):
    print(f"Can I run this code now to verify if it's correct?")
    print(event.data)
    res = input("Please input your answer (yes/no): ")
    print(f"\nPlease input your answer (yes/no): {res}\n")  # print the input
    if res in ["yes", "no"]:
        feedback_sender.send(Feedback(data=res))
    else:
        print("Invalid input. Please input yes or no.")
        feedback_sender.send(Feedback(data="no"))

# register can_run_code event handler to `CodeAssistant` automa
code_assistant = CodeAssistant()
code_assistant.register_event_handler("can_run_code", can_run_code_handler)

Now let's use it!

In [32]:
await code_assistant.arun(user_requirement="Please write a function to print 'Hello, World!'")

Can I run this code now to verify if it's correct?
```python
def print_hello_world():
    print('Hello, World!')

# Call the function to print 'Hello, World!'
print_hello_world()
```

Please input your answer (yes/no): yes

- - - - - - Result - - - - - -
Hello, World!
- - - - - - End - - - - - -


In the above example, Bridgic wrap the message sent to the human user in an [`Event`](../../../../reference/bridgic-core/bridgic/core/automa/interaction/#bridgic.core.automa.interaction.Event) and he message received from the user in a [`FeedBack`](../../../../reference/bridgic-core/bridgic/core/automa/interaction/#bridgic.core.automa.interaction.Feedback). 

- `Event` contains three fields:
    - `event_type`: A string. The event type is used to identify the registered event handler.
    - `timestamp`: A Python datetime object. The timestamp of the event. The default is `datetime.now()`.
    - `data`: The data attached to the event.
- `FeedBack` contains one field:
    - `data`: The data attached to the feedback.

`request_feedback_async()` send an event to the user and request for a feedback. This method call will block the caller until the feedback is received. However, thanks to Python’s asynchronous event loop mechanism, other automas running on the same main thread will not be blocked. 

The registered event handler must be defined as type of [`EventHandlerType`](../../../../reference/bridgic-core/bridgic/core/automa/interaction/#bridgic.core.automa.interaction.EventHandlerType).  Here it should be a function that takes an [`Event`](../../../../reference/bridgic-core/bridgic/core/automa/interaction/#bridgic.core.automa.interaction.Event) and a [`FeedbackSender`](../../../../reference/bridgic-core/bridgic/core/automa/interaction/#bridgic.core.automa.interaction.FeedbackSender) as arguments.

### Counting notifier

Sometimes, it may be necessary to post an event without expecting any feedback, for example, message notifications or progress updates. At this point, we call the [`post_event()`](../../../../reference/bridgic-core/bridgic/core/automa/#bridgic.core.automa.Automa.post_event) method and register a event handler of type [`EventHandlerType`](../../../../reference/bridgic-core/bridgic/core/automa/interaction/#bridgic.core.automa.interaction.EventHandlerType) to process the event. Here the event handler should be a function that takes only an `Event` as an argument

For example, a counting notifier is implemented to count from 1 up to the number specified by the `user_input` argument. The user can also specify which number (`notify_int`) should trigger a reminder.

In [ ]:
class MessageNotifier(GraphAutoma):
    @worker(is_start=True)
    async def notify(self, user_input: int, notify_int: int):
        print(f"Loop from 1 to {user_input}")
        for i in range(1, user_input + 1):
            if i == notify_int:
                event = Event(event_type="message_notification", data=f"Loop {i} times")
                self.post_event(event)

def message_notification_handler(event: Event):
    print(f'!! Now count to {event.data}. !!')

message_notifier = MessageNotifier()
message_notifier.register_event_handler("message_notification", message_notification_handler)
await message_notifier.arun(user_input=10, notify_int=5)
        

Loop from 1 to 10
!! Now count to Loop 5 times. !!


### Message assistant

In certain scenarios, it may be necessary to wait for feedback after triggering an event. However, since the feedback could take a long time to arrive, keeping the system in a waiting state would result in unnecessary resource consumption.

Bridgic provides a powerful [`interact_with_human`](../../../../reference/bridgic-core/bridgic/core/automa/#bridgic.core.automa.Automa.interact_with_human) mechanism for interruption recovery in this situation. This allows the program to pause and save its current execution state when such events occur, wait for feedback, and then resume execution.

Let's implement a message assistant that receives a message from user "A" and replies to it, but doesn't know how long it might have to wait for the reply.

In [ ]:
class MessageAssistant(GraphAutoma):
    @worker(is_start=True)
    async def receive_message(self, message: str):
        print(f'- - - - - - Received message - - - - - -')
        print(message)
        print(f'- - - - - - End - - - - - -\n')
        return message
    
    @worker(dependencies=["receive_message"])
    async def reply_message_and_wait_reply(self, message: str):
        print(f'- - - - - - Reply Message - - - - - -')
        response = await llm.achat(
            model=_model_name,
            messages=[
                Message.from_text(text=f"You are a message assistant. Please reply to the following message.", role=Role.SYSTEM),
                Message.from_text(text=message, role=Role.USER),
            ]
        )
        print(response.message.content)
        print(f'- - - - - - End - - - - - -\n')

        if "Bye!" in message:  # if the message contains "Bye!", the reply is complete
            return 

        # wait for reply            
        event = Event(event_type="wait_for_reply")
        feedback: InteractionFeedback = self.interact_with_human(event)  # interrupt to wait for reply and resume when feedback is received
        self.ferry_to("receive_message", feedback.data)

Calling [`interact_with_human()`](../../../../reference/bridgic-core/bridgic/core/automa/#bridgic.core.automa.Automa.interact_with_human) posts an event that might take an unpredictable amount of time to finish. Consequently, an [`InteractionException`](../../../../reference/bridgic-core/bridgic/core/automa/interaction/#bridgic.core.automa.interaction.InteractionException) is thrown by the [`arun`](../../../../reference/bridgic-core/bridgic/core/automa/#bridgic.core.automa.GraphAutoma.arun) method of the automa.

The `InteractionException` contains two fields:

- `interactions`: A list of [`Interaction`](../../../../reference/bridgic-core/bridgic/core/automa/interaction/#bridgic.core.automa.interaction.Interaction)s, each `Interaction` containing an `interaction_id` and an `event`.
- `snapshot`: a snapshot of the Automa's current state, of type [`Snapshot`](../../../../reference/bridgic-core/bridgic/core/automa/#bridgic.core.automa.Snapshot).

The snapshot corresponding to the interaction can be persisted in the database, and the execution can be resumed when needed in the future.

In [14]:
import tempfile
import os

# Use a temporary directory to achieve the purpose of persistent storage.
temp_dir = tempfile.TemporaryDirectory()

# Use a dictionary to record the interaction id and the corresponding snapshot path.
cache_dict = {}

# deal with delayed interaction
message_assistant = MessageAssistant()
try:
    await message_assistant.arun(message="Hello, how are you?")
except InteractionException as e:
    interaction_id = e.interactions[0].interaction_id
    bytes_file = os.path.join(temp_dir.name, f"message_assistant_{interaction_id}.bytes")
    version_file = os.path.join(temp_dir.name, f"message_assistant_{interaction_id}.version")
    with open(bytes_file, "wb") as f:
        f.write(e.snapshot.serialized_bytes)
    with open(version_file, "w") as f:
        f.write(e.snapshot.serialization_version)

    cache_dict["A"] = {
        "interaction_id": interaction_id,
        "bytes_file": bytes_file,
        "version_file": version_file
    }
    print(f"! ! ! State has been saved and can be resumed later. ! ! !")

- - - - - - Received message - - - - - -
Hello, how are you?
- - - - - - End - - - - - -

- - - - - - Reply Message - - - - - -
Hello! I'm functioning well, thank you for asking. I'm always excited to chat and help out! 😊 How can I assist you today?
- - - - - - End - - - - - -

! ! ! State has been saved and can be resumed later. ! ! !


Suppose after quite a long time, there was finally a reply.

In [ ]:
# build feedback
user = "A"
reply_message = "I really enjoy talking with you. Bye!"
interaction_id = cache_dict[user]["interaction_id"]
feedback = InteractionFeedback(
    interaction_id=interaction_id,
    data=reply_message
)

# load snapshot
bytes_file = cache_dict[user]["bytes_file"]
version_file = cache_dict[user]["version_file"]
with open(bytes_file, "rb") as f:
    serialized_bytes = f.read()
with open(version_file, "r") as f:
    serialization_version = f.read()
snapshot = Snapshot(
    serialized_bytes=serialized_bytes, 
    serialization_version=serialization_version
)

# automa resumes from the snapshot
message_assistant = MessageAssistant.load_from_snapshot(snapshot)
await message_assistant.arun(interaction_feedback=feedback)

- - - - - - Reply Message - - - - - -
Hello! I'm functioning well, thank you for asking. I'm always excited to chat and help out! 😊 How can I assist you today?
- - - - - - End - - - - - -

- - - - - - Received message - - - - - -
I really enjoy taking with you. Bye!
- - - - - - End - - - - - -

- - - - - - Reply Message - - - - - -
Thank you for enjoying our conversation! I'm glad I could help. Have a wonderful day, and take care! 😊✨
- - - - - - End - - - - - -



When facing a situation that requires feedback but the waiting time is uncertain, this mechanism saves the current state and re-enters when the right moment comes in the future. This not only enables the system to release resources that would otherwise be occupied for a long time, but also allows it to be awakened at an appropriate time.

## What have we done

No matter which form of human-in-the-loop it is, Bridgic provides flexible support.

- [`request_feedback_async`](../../../../reference/bridgic-core/bridgic/core/automa/#bridgic.core.automa.Automa.request_feedback_async): Used when the event must return feedback before the program can proceed. The program remains blocked until feedback is received.
- [`post_event`](../../../../reference/bridgic-core/bridgic/core/automa/#bridgic.core.automa.Automa.post_event): Used when you just want to notify or trigger an event without expecting any feedback. The main program never blocks.
- [`interact_with_human`](../../../../reference/bridgic-core/bridgic/core/automa/#bridgic.core.automa.Automa.interact_with_human): Used when feedback is required but may arrive much later. The program is suspended and persisted, and resumes immediately when feedback becomes available.